In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#load modules
import numpy as np
np.set_printoptions(suppress=True) # scientific notation gets annoying
import matplotlib.pyplot as plt
import matplotlib
from astropy import units as u
from astropy.io import fits
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.ndimage
import random

# PSD code
import h5py
import copy
import os

# import personal code
from model_kit import psd_functions as psd
from model_kit import datafiles as dfx
from model_kit import mvls

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from datetime import datetime
date_ran = datetime.now()
dr_str = date_ran.strftime("%d/%m/%Y %H:%M:%S")
print('Last date this notebook has been ran: {0}'.format(dr_str))

Last date this notebook has been ran: 20/09/2021 18:35:18


In [10]:
def mvls_cal(surf_var, psd, dk):
    psd_raw = psd*(dk**2) # resets the psd
    psd_norm = psd_raw/(np.sum(psd_raw)*(dk**2))
    psd_cal = psd_norm * surf_var
    return psd_cal

def cal_loop(file_dir, surf_name, surf_unit, mask_name, 
             psd_dir, psd_name, psd_unit):
    # open up the data
    opt_fits = fits.open(file_dir+surf_name)[0]
    opt_data = (opt_fits.data * surf_unit).to(u.nm)
    dx = opt_fits.header['LATRES']*u.m # not sure if needed
    
    # open the mask
    opt_mask = fits.open(file_dir+mask_name)[0].data
    
    # calculate the surface variance
    surf_var = np.var(opt_data[opt_mask==1]) # should have opt_data.unit**2
    
    # open the PSD
    psd_fits = fits.open(psd_dir+psd_name+'.fits')[0]
    psd_data = psd_fits.data * psd_unit
    psd_hdr = psd_fits.header
    dk = psd_hdr['delta_k'] / u.m
    
    # apply psd calibration
    psd_cal = mvls_cal(surf_var=surf_var, psd=psd_data, dk=dk)
    
    # calculate some quick values
    kmin = dk
    kmax = int(psd_cal.shape[0]/2)*dk
    psd_rms = psd.do_psd_rms(psd_data=psd_cal, delta_k=dk,
                             k_tgt_lim=(kmin*0.5, kmax))
    # scale down kmin by 0.5 because 
    print('surf_rms={0:.3f}, psd_rms={1:.3f}'.format(np.sqrt(surf_var), psd_rms))
    # add in header content
    psd_hdr['surf_var'] = (surf_var.value, 
                           'Surface variance [{0}]'.format(surf_var.unit))
    psd_hdr['kmin'] = (kmin.value,
                       'Minimum spatial frequency limit [{0}]'.format(kmin.unit))
    psd_hdr['kmax'] = (kmax.value,
                       'Maximum spatial frequency limit [{0}]'.format(kmax.unit))
    psd_hdr['psd_rms'] = (psd_rms.value,
                       'RMS calculated from PSD [{0}]'.format(psd_rms.unit))
    
    # save the data
    fits.writeto(psd_dir+psd_name+'_cal.fits', psd_cal.value,
                 psd_hdr, overwrite=True)

In [11]:
# 1-inch flats, unglued
file_dir = 'flat_mirrors/fixed_fits/'
psd_folder = 'flat_mirrors/mvls_psd/'
tot_fm = 8
tot_step = 4
nt = 3
flat_label = '1in'

for nfm in range(0, tot_fm):
    for nstep in range(0, tot_step):
        surf_name = 'flat_{0}_n{1}_80CA_step{2}_z{3}_surf.fits'.format(flat_label,
                                                                       nfm+1, nstep, nt)
        surf_unit = u.micron
        mask_name = 'flat_{0}_n{1}_80CA_step{2}_bigdust_mask.fits'.format(flat_label,
                                                                          nfm+1, nstep)
        psd_name = 'lspsd_fm_{0}_n{1}_step{2}_z{3}'.format(flat_label, nfm+1, nstep, nt)
        psd_unit = (u.nm*u.m)**2
        cal_loop(file_dir=file_dir, surf_name=surf_name, surf_unit=surf_unit, 
                 mask_name=mask_name, psd_dir=psd_folder, psd_name=psd_name, 
                 psd_unit=psd_unit)
    print('PSD calibrated for n{0}'.format(nfm+1))

surf_rms=2.988 nm, psd_rms=2.670 nm
surf_rms=3.379 nm, psd_rms=3.015 nm
surf_rms=2.973 nm, psd_rms=2.659 nm
surf_rms=2.789 nm, psd_rms=2.500 nm
PSD calibrated for n1
surf_rms=3.097 nm, psd_rms=2.772 nm
surf_rms=3.165 nm, psd_rms=2.830 nm
surf_rms=2.922 nm, psd_rms=2.614 nm
surf_rms=2.935 nm, psd_rms=2.629 nm
PSD calibrated for n2
surf_rms=3.390 nm, psd_rms=3.029 nm
surf_rms=3.518 nm, psd_rms=3.142 nm
surf_rms=3.238 nm, psd_rms=2.895 nm
surf_rms=3.287 nm, psd_rms=2.941 nm
PSD calibrated for n3
surf_rms=2.482 nm, psd_rms=2.215 nm
surf_rms=2.704 nm, psd_rms=2.415 nm
surf_rms=2.508 nm, psd_rms=2.245 nm
surf_rms=2.554 nm, psd_rms=2.285 nm
PSD calibrated for n4
surf_rms=2.522 nm, psd_rms=2.256 nm
surf_rms=2.645 nm, psd_rms=2.364 nm
surf_rms=2.623 nm, psd_rms=2.351 nm
surf_rms=2.630 nm, psd_rms=2.357 nm
PSD calibrated for n5
surf_rms=2.856 nm, psd_rms=2.556 nm
surf_rms=3.329 nm, psd_rms=2.976 nm
surf_rms=3.334 nm, psd_rms=2.982 nm
surf_rms=3.102 nm, psd_rms=2.778 nm
PSD calibrated for n6
surf

In [12]:
# 2-inch flats, unglued
file_dir = 'flat_mirrors/fixed_fits/'
psd_folder = 'flat_mirrors/mvls_psd/'
tot_fm = 8
tot_step = 4
nt = 3
flat_label = '2in'

for nfm in range(0, tot_fm):
    for nstep in range(0, tot_step):
        surf_name = 'flat_{0}_n{1}_80CA_step{2}_z{3}_surf.fits'.format(flat_label,
                                                                       nfm+1, nstep, nt)
        surf_unit = u.micron
        mask_name = 'flat_{0}_n{1}_80CA_step{2}_bigdust_mask.fits'.format(flat_label,
                                                                          nfm+1, nstep)
        psd_name = 'lspsd_fm_{0}_n{1}_step{2}_z{3}'.format(flat_label, nfm+1, nstep, nt)
        psd_unit = (u.nm*u.m)**2
        cal_loop(file_dir=file_dir, surf_name=surf_name, surf_unit=surf_unit, 
                 mask_name=mask_name, psd_dir=psd_folder, psd_name=psd_name, 
                 psd_unit=psd_unit)
    print('PSD calibrated for n{0}'.format(nfm+1))

surf_rms=2.530 nm, psd_rms=2.300 nm
surf_rms=2.964 nm, psd_rms=2.689 nm
surf_rms=2.573 nm, psd_rms=2.341 nm
surf_rms=1.896 nm, psd_rms=1.727 nm
PSD calibrated for n1
surf_rms=4.035 nm, psd_rms=3.644 nm
surf_rms=4.108 nm, psd_rms=3.697 nm
surf_rms=4.248 nm, psd_rms=3.839 nm
surf_rms=4.339 nm, psd_rms=3.917 nm
PSD calibrated for n2
surf_rms=1.255 nm, psd_rms=1.249 nm
surf_rms=1.656 nm, psd_rms=1.521 nm
surf_rms=1.361 nm, psd_rms=1.360 nm
surf_rms=1.346 nm, psd_rms=1.341 nm
PSD calibrated for n3
surf_rms=4.947 nm, psd_rms=4.376 nm
surf_rms=4.702 nm, psd_rms=4.173 nm
surf_rms=5.583 nm, psd_rms=4.951 nm
surf_rms=4.802 nm, psd_rms=4.254 nm
PSD calibrated for n4
surf_rms=4.560 nm, psd_rms=3.966 nm
surf_rms=4.621 nm, psd_rms=4.024 nm
surf_rms=4.590 nm, psd_rms=3.992 nm
surf_rms=4.840 nm, psd_rms=4.214 nm
PSD calibrated for n5
surf_rms=3.174 nm, psd_rms=2.819 nm
surf_rms=2.900 nm, psd_rms=2.705 nm
surf_rms=3.854 nm, psd_rms=3.438 nm
surf_rms=2.969 nm, psd_rms=2.635 nm
PSD calibrated for n6
surf

In [13]:
# 1-inch flats, glued
file_dir = 'flat_mirrors/fixed_fits/'
psd_folder = 'flat_mirrors/mvls_psd/'
tot_fm = 1
tot_step = 4
flat_label = '1in'

for nfm in range(0, tot_fm):
    for nstep in range(0, tot_step):
        surf_name = 'flat_{0}_n{1}_80CA_step{2}_glue_surf.fits'.format(flat_label,
                                                                       nfm+1, nstep)
        surf_unit = u.micron
        mask_name = 'flat_{0}_n{1}_80CA_step{2}_bigdust_mask.fits'.format(flat_label,
                                                                          nfm+1, nstep)
        psd_name = 'lspsd_fm_{0}_n{1}_step{2}_glue'.format(flat_label, nfm+1, nstep)
        psd_unit = (u.nm*u.m)**2
        cal_loop(file_dir=file_dir, surf_name=surf_name, surf_unit=surf_unit, 
                 mask_name=mask_name, psd_dir=psd_folder, psd_name=psd_name, 
                 psd_unit=psd_unit)
    print('PSD calibrated for n{0}'.format(nfm+1))

surf_rms=19.883 nm, psd_rms=18.219 nm
surf_rms=20.072 nm, psd_rms=18.378 nm
surf_rms=19.736 nm, psd_rms=18.157 nm
surf_rms=19.234 nm, psd_rms=17.652 nm
PSD calibrated for n1


In [14]:
# 0.5-inch flats, glued
file_dir = 'flat_mirrors/fixed_fits/'
psd_folder = 'flat_mirrors/mvls_psd/'
tot_fm = 1
tot_step = 4
flat_label = '0p5in'

for nfm in range(0, tot_fm):
    for nstep in range(0, tot_step):
        surf_name = 'flat_{0}_n{1}_80CA_step{2}_glue_surf.fits'.format(flat_label,
                                                                       nfm+1, nstep)
        surf_unit = u.micron
        mask_name = 'flat_{0}_n{1}_80CA_step{2}_bigdust_mask.fits'.format(flat_label,
                                                                          nfm+1, nstep)
        psd_name = 'lspsd_fm_{0}_n{1}_step{2}_glue'.format(flat_label, nfm+1, nstep)
        psd_unit = (u.nm*u.m)**2
        cal_loop(file_dir=file_dir, surf_name=surf_name, surf_unit=surf_unit, 
                 mask_name=mask_name, psd_dir=psd_folder, psd_name=psd_name, 
                 psd_unit=psd_unit)
    print('PSD calibrated for n{0}'.format(nfm+1))

surf_rms=18.920 nm, psd_rms=17.713 nm
surf_rms=18.471 nm, psd_rms=17.422 nm
surf_rms=19.051 nm, psd_rms=17.795 nm
surf_rms=18.728 nm, psd_rms=17.525 nm
PSD calibrated for n1
